In [29]:
from collections import deque

class BacktrackingMapColoring:
    def __init__(self, regions, adjacencies, colors):
        self.regions = regions
        self.adjacencies = adjacencies
        self.colors = colors
        self.domains = {region: list(colors) for region in regions}
        self.assignments = {}

    def is_consistent(self, region, color):
        for neighbor in self.adjacencies.get(region, []):
            if neighbor in self.assignments and self.assignments[neighbor] == color:
                return False
        return True

    def select_region(self):
        unassigned_regions = [r for r in self.regions if r not in self.assignments]
        region = min(unassigned_regions, key=lambda r: len(self.domains[r]))
        print(f"Selected region: {region} with domain {self.domains[region]}")
        return region

    def backtrack(self):
        print(f"Current assignments: {self.assignments}")
        if len(self.assignments) == len(self.regions):
            return True

        region = self.select_region()
        for color in self.domains[region]:
            print(f"Trying {color} for {region}")
            if self.is_consistent(region, color):
                self.assignments[region] = color
                if self.backtrack():
                    return True
                self.assignments.pop(region)
                print(f"Backtracking from {region}, removing color {color}")

        return False

    def solve(self):
        print("Starting Backtracking...")
        if self.backtrack():
            print("Solution found!")
            return self.assignments
        else:
            print("No solution found.")
            return None

class AC3MapColoring:
    def __init__(self, regions, adjacencies, colors):
        self.regions = regions
        self.adjacencies = adjacencies
        self.colors = colors
        self.domains = {region: list(colors) for region in regions}
        self.assignments = {}

    def is_consistent(self, region, color):
        for neighbor in self.adjacencies.get(region, []):
            if neighbor in self.assignments and self.assignments[neighbor] == color:
                return False
        return True

    def ac3(self):
        queue = deque([(region, neighbor) for region in self.regions for neighbor in self.adjacencies.get(region, [])])
        print(f"Initial AC-3 queue: {list(queue)}")
        while queue:
            xi, xj = queue.popleft()
            print(f"Processing arc ({xi}, {xj})")
            if self.revise(xi, xj):
                if len(self.domains[xi]) == 0:
                    return False
                for xk in self.adjacencies[xi]:
                    if xk != xj:
                        queue.append((xk, xi))
            print(f"Domains after processing arc ({xi}, {xj}): {self.domains}")
        return True

    def revise(self, xi, xj):
        revised = False
        for x in self.domains[xi][:]:
            if not any(self.is_consistent_with(x, y) for y in self.domains[xj]):
                print(f"Removing {x} from domain of {xi} due to conflict with {xj}")
                self.domains[xi].remove(x)
                revised = True
        return revised

    def is_consistent_with(self, x, y):
        return x != y

    def select_region(self):
        unassigned_regions = [r for r in self.regions if r not in self.assignments]
        region = min(unassigned_regions, key=lambda r: len(self.domains[r]))
        print(f"Selected region: {region} with domain {self.domains[region]}")
        return region

    def backtrack(self):
        print(f"Current assignments: {self.assignments}")
        if len(self.assignments) == len(self.regions):
            return True

        region = self.select_region()
        for color in self.domains[region]:
            print(f"Trying {color} for {region}")
            if self.is_consistent(region, color):
                self.assignments[region] = color
                if self.ac3():
                    print(f"AC-3 consistent for {region} with color {color}")
                    if self.backtrack():
                        return True
                self.assignments.pop(region)
                print(f"Backtracking from {region}, removing color {color}")

        return False

    def solve(self):
        print("Starting AC-3 with Backtracking...")
        if self.ac3() and self.backtrack():
            print("Solution found!")
            return self.assignments
        else:
            print("No solution found.")
            return None

class ForwardCheckingMapColoring:
    def __init__(self, regions, adjacencies, colors):
        self.regions = regions
        self.adjacencies = adjacencies
        self.colors = colors
        self.domains = {region: list(colors) for region in regions}
        self.assignments = {}

    def is_consistent(self, region, color):
        for neighbor in self.adjacencies.get(region, []):
            if neighbor in self.assignments and self.assignments[neighbor] == color:
                return False
        return True

    def forward_checking(self, region, color):
        saved_domains = {region: list(self.domains[region]) for region in self.regions}
        print(f"Forward checking for {region} with color {color}")
        for neighbor in self.adjacencies.get(region, []):
            if neighbor not in self.assignments and color in self.domains[neighbor]:
                self.domains[neighbor].remove(color)
                if not self.domains[neighbor]:
                    print(f"Domain of {neighbor} is empty after removing {color}")
                    return False, saved_domains
        print(f"Domains after forward checking: {self.domains}")
        return True, saved_domains

    def select_region(self):
        unassigned_regions = [r for r in self.regions if r not in self.assignments]
        region = min(unassigned_regions, key=lambda r: len(self.domains[r]))
        print(f"Selected region: {region} with domain {self.domains[region]}")
        return region

    def backtrack(self):
        print(f"Current assignments: {self.assignments}")
        if len(self.assignments) == len(self.regions):
            return True

        region = self.select_region()
        for color in self.domains[region]:
            print(f"Trying {color} for {region}")
            if self.is_consistent(region, color):
                self.assignments[region] = color
                consistent, saved_domains = self.forward_checking(region, color)
                if consistent:
                    print(f"Forward checking successful for {region} with color {color}")
                    if self.backtrack():
                        return True
                self.assignments.pop(region)
                self.domains = saved_domains
                print(f"Backtracking from {region}, removing color {color}")

        return False

    def solve(self):
        print("Starting Forward Checking with Backtracking...")
        if self.backtrack():
            print("Solution found!")
            return self.assignments
        else:
            print("No solution found.")
            return None


# Example usage
regions = ['A', 'B', 'C', 'D', 'E']
adjacencies = {
    'A': ['B', 'C'],
    'B': ['A', 'C', 'D'],
    'C': ['A', 'B', 'D', 'E'],
    'D': ['B', 'C', 'E'],
    'E': ['C', 'D']
}
colors = ['Red', 'Green', 'Blue']

# Backtracking
print("Backtracking Map Coloring")
solver = BacktrackingMapColoring(regions, adjacencies, colors)
solution = solver.solve()
print("\nFinal Solution:")
if solution:
    for region, color in solution.items():
        print(f"{region}: {color}")
else:
    print("No solution found.")

# AC-3
print("\nAC-3 Map Coloring")
solver = AC3MapColoring(regions, adjacencies, colors)
solution = solver.solve()
print("\nFinal Solution:")
if solution:
    for region, color in solution.items():
        print(f"{region}: {color}")
else:
    print("No solution found.")

# Forward Checking
print("\nForward Checking Map Coloring")
solver = ForwardCheckingMapColoring(regions, adjacencies, colors)
solution = solver.solve()
print("\nFinal Solution:")
if solution:
    for region, color in solution.items():
        print(f"{region}: {color}")
else:
    print("No solution found.")


Backtracking Map Coloring
Starting Backtracking...
Current assignments: {}
Selected region: A with domain ['Red', 'Green', 'Blue']
Trying Red for A
Current assignments: {'A': 'Red'}
Selected region: B with domain ['Red', 'Green', 'Blue']
Trying Red for B
Trying Green for B
Current assignments: {'A': 'Red', 'B': 'Green'}
Selected region: C with domain ['Red', 'Green', 'Blue']
Trying Red for C
Trying Green for C
Trying Blue for C
Current assignments: {'A': 'Red', 'B': 'Green', 'C': 'Blue'}
Selected region: D with domain ['Red', 'Green', 'Blue']
Trying Red for D
Current assignments: {'A': 'Red', 'B': 'Green', 'C': 'Blue', 'D': 'Red'}
Selected region: E with domain ['Red', 'Green', 'Blue']
Trying Red for E
Trying Green for E
Current assignments: {'A': 'Red', 'B': 'Green', 'C': 'Blue', 'D': 'Red', 'E': 'Green'}
Solution found!

Final Solution:
A: Red
B: Green
C: Blue
D: Red
E: Green

AC-3 Map Coloring
Starting AC-3 with Backtracking...
Initial AC-3 queue: [('A', 'B'), ('A', 'C'), ('B', 'A')

In [28]:
from collections import deque

class BacktrackingMapColoring:
    def __init__(self, regions, adjacencies, colors):
        self.regions = regions
        self.adjacencies = adjacencies
        self.colors = colors
        self.domains = {region: list(colors) for region in regions}
        self.assignments = {}

    def is_consistent(self, region, color):
        for neighbor in self.adjacencies.get(region, []):
            if neighbor in self.assignments and self.assignments[neighbor] == color:
                return False
        return True

    def least_constraining_value(self, region):
        sorted_colors = sorted(self.domains[region], key=lambda color: self.count_constraints(region, color))
        return sorted_colors

    def count_constraints(self, region, color):
        count = 0
        for neighbor in self.adjacencies.get(region, []):
            if neighbor not in self.assignments and color in self.domains[neighbor]:
                count += 1
        return count

    def select_region(self):
        unassigned_regions = [r for r in self.regions if r not in self.assignments]
        return min(unassigned_regions, key=lambda r: len(self.domains[r]))

    def print_state(self, action, region, color=None):
        print(f"Action: {action}")
        print(f"Region: {region}")
        if color:
            print(f"Color: {color}")
        print(f"Assignments: {self.assignments}")
        print(f"Domains: {self.domains}")
        print("-" * 40)

    def backtrack(self):
        if len(self.assignments) == len(self.regions):
            return True

        region = self.select_region()
        for color in self.least_constraining_value(region):
            self.print_state("Trying", region, color)
            if self.is_consistent(region, color):
                self.assignments[region] = color
                if self.backtrack():
                    return True
                self.assignments.pop(region)
                self.print_state("Backtracking from", region, color)
                
        return False

    def solve(self):
        if self.backtrack():
            return self.assignments
        else:
            return None

class AC3MapColoring:
    def __init__(self, regions, adjacencies, colors):
        self.regions = regions
        self.adjacencies = adjacencies
        self.colors = colors
        self.domains = {region: list(colors) for region in regions}
        self.assignments = {}

    def is_consistent(self, region, color):
        for neighbor in self.adjacencies.get(region, []):
            if neighbor in self.assignments and self.assignments[neighbor] == color:
                return False
        return True

    def ac3(self):
        queue = deque([(region, neighbor) for region in self.regions for neighbor in self.adjacencies.get(region, [])])
        while queue:
            xi, xj = queue.popleft()
            if self.revise(xi, xj):
                if len(self.domains[xi]) == 0:
                    return False
                for xk in self.adjacencies[xi]:
                    if xk != xj:
                        queue.append((xk, xi))
        return True

    def revise(self, xi, xj):
        revised = False
        for x in self.domains[xi][:]:
            if not any(self.is_consistent_with(x, y) for y in self.domains[xj]):
                print(f"Removing {x} from domain of {xi} due to conflict with {xj}")
                self.domains[xi].remove(x)
                revised = True
        return revised

    def is_consistent_with(self, x, y):
        return x != y

    def least_constraining_value(self, region):
        sorted_colors = sorted(self.domains[region], key=lambda color: self.count_constraints(region, color))
        return sorted_colors

    def count_constraints(self, region, color):
        count = 0
        for neighbor in self.adjacencies.get(region, []):
            if neighbor not in self.assignments and color in self.domains[neighbor]:
                count += 1
        return count

    def select_region(self):
        unassigned_regions = [r for r in self.regions if r not in self.assignments]
        return min(unassigned_regions, key=lambda r: len(self.domains[r]))

    def print_state(self, action, region, color=None):
        print(f"Action: {action}")
        print(f"Region: {region}")
        if color:
            print(f"Color: {color}")
        print(f"Assignments: {self.assignments}")
        print(f"Domains: {self.domains}")
        print("-" * 40)

    def backtrack(self):
        if len(self.assignments) == len(self.regions):
            return True

        region = self.select_region()
        for color in self.least_constraining_value(region):
            self.print_state("Trying", region, color)
            if self.is_consistent(region, color):
                self.assignments[region] = color
                if self.ac3():
                    print(f"AC-3 consistent for {region} with color {color}")
                    if self.backtrack():
                        return True
                self.assignments.pop(region)
                self.print_state("Backtracking from", region, color)

        return False

    def solve(self):
        if self.ac3() and self.backtrack():
            return self.assignments
        else:
            return None

class ForwardCheckingMapColoring:
    def __init__(self, regions, adjacencies, colors):
        self.regions = regions
        self.adjacencies = adjacencies
        self.colors = colors
        self.domains = {region: list(colors) for region in regions}
        self.assignments = {}

    def is_consistent(self, region, color):
        for neighbor in self.adjacencies.get(region, []):
            if neighbor in self.assignments and self.assignments[neighbor] == color:
                return False
        return True

    def forward_checking(self, region, color):
        saved_domains = {region: list(self.domains[region]) for region in self.regions}
        for neighbor in self.adjacencies.get(region, []):
            if neighbor not in self.assignments and color in self.domains[neighbor]:
                self.domains[neighbor].remove(color)
                if not self.domains[neighbor]:
                    return False, saved_domains
        return True, saved_domains

    def least_constraining_value(self, region):
        sorted_colors = sorted(self.domains[region], key=lambda color: self.count_constraints(region, color))
        return sorted_colors

    def count_constraints(self, region, color):
        count = 0
        for neighbor in self.adjacencies.get(region, []):
            if neighbor not in self.assignments and color in self.domains[neighbor]:
                count += 1
        return count

    def select_region(self):
        unassigned_regions = [r for r in self.regions if r not in self.assignments]
        return min(unassigned_regions, key=lambda r: len(self.domains[r]))

    def print_state(self, action, region, color=None):
        print(f"Action: {action}")
        print(f"Region: {region}")
        if color:
            print(f"Color: {color}")
        print(f"Assignments: {self.assignments}")
        print(f"Domains: {self.domains}")
        print("-" * 40)

    def backtrack(self):
        if len(self.assignments) == len(self.regions):
            return True

        region = self.select_region()
        for color in self.least_constraining_value(region):
            self.print_state("Trying", region, color)
            if self.is_consistent(region, color):
                self.assignments[region] = color
                consistent, saved_domains = self.forward_checking(region, color)
                if consistent:
                    print(f"Forward checking successful for {region} with color {color}")
                    if self.backtrack():
                        return True
                self.assignments.pop(region)
                self.domains = saved_domains
                self.print_state("Backtracking from", region, color)

        return False

    def solve(self):
        if self.backtrack():
            return self.assignments
        else:
            return None

# Example usage
regions = ['A', 'B', 'C', 'D', 'E']
adjacencies = {
    'A': ['B', 'C'],
    'B': ['A', 'C', 'D'],
    'C': ['A', 'B', 'D', 'E'],
    'D': ['B', 'C', 'E'],
    'E': ['C', 'D']
}
colors = ['Red', 'Green', 'Blue']

# Backtracking
print("Backtracking Map Coloring")
solver = BacktrackingMapColoring(regions, adjacencies, colors)
solution = solver.solve()
print("\nFinal Solution:")
if solution:
    for region, color in solution.items():
        print(f"{region}: {color}")
else:
    print("No solution found.")

# AC-3
print("\nAC-3 Map Coloring")
solver = AC3MapColoring(regions, adjacencies, colors)
solution = solver.solve()
print("\nFinal Solution:")
if solution:
    for region, color in solution.items():
        print(f"{region}: {color}")
else:
    print("No solution found.")

# Forward Checking
print("\nForward Checking Map Coloring")
solver = ForwardCheckingMapColoring(regions, adjacencies, colors)
solution = solver.solve()
print("\nFinal Solution:")
if solution:
    for region, color in solution.items():
        print(f"{region}: {color}")
else:
    print("No solution found.")


Backtracking Map Coloring
Action: Trying
Region: A
Color: Red
Assignments: {}
Domains: {'A': ['Red', 'Green', 'Blue'], 'B': ['Red', 'Green', 'Blue'], 'C': ['Red', 'Green', 'Blue'], 'D': ['Red', 'Green', 'Blue'], 'E': ['Red', 'Green', 'Blue']}
----------------------------------------
Action: Trying
Region: B
Color: Red
Assignments: {'A': 'Red'}
Domains: {'A': ['Red', 'Green', 'Blue'], 'B': ['Red', 'Green', 'Blue'], 'C': ['Red', 'Green', 'Blue'], 'D': ['Red', 'Green', 'Blue'], 'E': ['Red', 'Green', 'Blue']}
----------------------------------------
Action: Trying
Region: B
Color: Green
Assignments: {'A': 'Red'}
Domains: {'A': ['Red', 'Green', 'Blue'], 'B': ['Red', 'Green', 'Blue'], 'C': ['Red', 'Green', 'Blue'], 'D': ['Red', 'Green', 'Blue'], 'E': ['Red', 'Green', 'Blue']}
----------------------------------------
Action: Trying
Region: C
Color: Red
Assignments: {'A': 'Red', 'B': 'Green'}
Domains: {'A': ['Red', 'Green', 'Blue'], 'B': ['Red', 'Green', 'Blue'], 'C': ['Red', 'Green', 'Blue'],